<a href="https://colab.research.google.com/github/AnikaitLakhotia/Machine-Learning/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import io
import os

In [4]:
from google.colab import files
data = files.upload()

Saving creditcard.csv to creditcard (1).csv


In [5]:
data_frame = pd.read_csv(io.StringIO(data['creditcard.csv'].decode('utf-8')))

In [6]:
features = data_frame.iloc[:, :-1].values
targets = data_frame.iloc[:, -1].values

In [7]:
# Read Dataset
data_frame.head()

Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [8]:
# Check for any null values
data_frame.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [31]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [32]:
# Give class weights
class_weights = {
    0: np.sum(targets == 1) / len(targets),
    1: np.sum(targets == 0) / len(targets)
}

In [33]:
# Normalize data
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
stand_dev = np.std(train_features, axis=0)
train_features /= stand_dev
val_features /= stand_dev

In [34]:
# Model architecture
model = keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [48]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20,
 restore_best_weights=True)

ReduceOnPlateau = keras.callbacks.ReduceLROnPlateau(
monitor='val_loss',
factor=0.1,
patience=4,
)

root_logdir = os.path.join(os.curdir, "my_logs2")
def get_run_logdir():
 import time
 run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
 return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [49]:
# metrics and compilation
metrics = [
    keras.metrics.FalseNegatives(name="false neg"),
    keras.metrics.FalsePositives(name="false pos"),
    keras.metrics.TrueNegatives(name="true neg"),
    keras.metrics.TruePositives(name="true pos"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

In [51]:
# Train model
model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=50,
    verbose=2,
    validation_data=(val_features, val_targets),
    class_weight=class_weights,
    callbacks=[early_stopping_cb, ReduceOnPlateau]
)

Epoch 1/50
112/112 - 1s - loss: 1.4995e-04 - false neg: 1.0000 - false pos: 4840.0000 - true neg: 222589.0000 - true pos: 416.0000 - precision: 0.0791 - recall: 0.9976 - val_loss: 0.0280 - val_false neg: 11.0000 - val_false pos: 414.0000 - val_true neg: 56472.0000 - val_true pos: 64.0000 - val_precision: 0.1339 - val_recall: 0.8533 - lr: 1.0000e-08 - 764ms/epoch - 7ms/step
Epoch 2/50
112/112 - 1s - loss: 1.4654e-04 - false neg: 2.0000 - false pos: 4808.0000 - true neg: 222621.0000 - true pos: 415.0000 - precision: 0.0795 - recall: 0.9952 - val_loss: 0.0280 - val_false neg: 11.0000 - val_false pos: 416.0000 - val_true neg: 56470.0000 - val_true pos: 64.0000 - val_precision: 0.1333 - val_recall: 0.8533 - lr: 1.0000e-08 - 791ms/epoch - 7ms/step
Epoch 3/50
112/112 - 1s - loss: 1.6515e-04 - false neg: 3.0000 - false pos: 4853.0000 - true neg: 222576.0000 - true pos: 414.0000 - precision: 0.0786 - recall: 0.9928 - val_loss: 0.0279 - val_false neg: 11.0000 - val_false pos: 414.0000 - val_true